### 1. Imports

In [6]:
import torch
from torch.utils.model_zoo import load_url
from PIL import Image
import matplotlib.pyplot as plt
from architectures import fornet,weights
from isplutils import utils
import random
import os
import cv2
from sklearn.metrics import precision_recall_fscore_support as score

### 2. Define global variables

In [5]:
net_model = 'EfficientNetAutoAttB4'
device = torch.device('cuda:5') if torch.cuda.is_available() else torch.device('cpu')
train_db = 'DFDC'
model_url = weights.weight_url['{:s}_{:s}'.format(net_model,train_db)]


### 3. Define and load model

In [22]:
net = getattr(fornet,net_model)().eval().to(device)
net.load_state_dict(load_url(model_url,map_location=device,check_hash=True))

Loaded pretrained weights for efficientnet-b4


<All keys matched successfully>

### 4. Define transform and test function

In [23]:
transf = utils.get_transformer('scale', 128, net.get_normalizer(), train=False)

In [62]:
def detector(net, x):
    net = net.to(device)
    return torch.sigmoid(
        net(
        (transf(image=j)['image']).unsqueeze(0)
                             .to(device))
        )  

### 5. Evaluate model

In [64]:
# path to the folder containing the images
folder_path = '/home/data/celeba/images'

# get a list of all the image file names in the folder
file_list = [f for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')]

# set random seed to make code reproducible
random.seed(123)

# generate 100 random indices between 0 and the number of images in the folder
rand_indices = random.sample(range(len(file_list)), 100)

# use the random indices to select the images
rand_images = [file_list[i] for i in rand_indices]

# load the trained model
model = net# load your trained model here

# create empty lists to store the true labels and predicted labels
true_labels = []
predicted_labels = []

# iterate through the selected images
for image in rand_images:
    # read the image file
    input = cv2.imread(os.path.join(folder_path, image))
    # preprocess the image as required by the model
    # e.g., resize, normalize, etc.
    # use the model to predict the label for the image
    prob = detector(model, input)
    if prob < 0.5:
        true_label = 0
    else:
        true_label = 1
    # get the true label for the image (e.g., from file name or metadata)
    # get the true label here
    # append the true and predicted labels to the respective lists
    true_labels.append(true_label)
    predicted_labels.append(0)

# use the sklearn.metrics.precision_recall_fscore_support function to calculate the precision, recall and F1-score
precision, recall, f1, _ = score(true_labels, predicted_labels, average='macro', zero_division=1)

# print the results
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1)


Precision:  1.0
Recall:  1.0
F1-score:  1.0
